<a href="https://colab.research.google.com/github/manojach87/miscellaneous/blob/master/coingecko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.coingecko.com/api/documentations/v3#/operations/ping

In [2]:
def readJsonLink(link):
  import urllib.request, json 
  with urllib.request.urlopen(link) as url:
      data = json.loads(url.read().decode())
      return data

In [3]:
def getMarkets(url="https://api.coingecko.com/api/v3/coins/markets?vs_currency=USD&order=market_cap_desc&per_page=250&sparkline=false",pages=10):
  marketsData=list()
  for i in range(1,pages):
    #print(i)
    marketsData.append(readJsonLink(url+"&page="+str(i)))
  return marketsData
#marketsData

In [4]:
def createDf(url,pages=10):
  import pandas as pd
  marketsData=getMarkets(url,pages=pages+1)
  marketsDf=pd.DataFrame()
  for i in range(0,len(marketsData)):
    #print(i)
    marketsDf=marketsDf.append(pd.DataFrame(marketsData[i]))
  marketsDf=marketsDf.reset_index()
  return marketsDf

In [5]:
marketsDf=createDf(url="https://api.coingecko.com/api/v3/coins/markets?vs_currency=USD&order=market_cap_desc&per_page=250&sparkline=false",pages=28)

In [6]:
#marketsDf[marketsDf.index==0]
marketsDf

,index,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,56349.00,1.055758e+12,1,1.186508e+12,1.020319e+11,60424.00,52810.000000,-4051.697236,-6.70798,-7.793897e+10,-6.87477,1.868586e+07,2.100000e+07,21000000.0,64805.00,-12.91377,2021-04-14T11:54:46.763Z,67.810000,8.312788e+04,2013-07-06T00:00:00.000Z,None,2021-04-18T23:31:31.131Z
1,1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,2236.34,2.589238e+11,2,NaN,6.078620e+10,2358.72,1970.000000,-122.373113,-5.18812,-1.581182e+10,-5.75529,1.155253e+08,NaN,NaN,2544.30,-11.94054,2021-04-16T01:00:14.650Z,0.432979,5.173621e+05,2015-10-20T00:00:00.000Z,"{'times': 52.03349336842597, 'currency': 'btc'...",2021-04-18T23:30:19.827Z
2,2,binancecoin,bnb,Binance Coin,https://assets.coingecko.com/coins/images/825/...,477.02,7.407571e+10,3,8.174531e+10,6.711831e+09,524.37,432.020000,-47.341592,-9.02835,-7.507768e+09,-9.20256,1.545337e+08,1.705337e+08,170533651.9,610.06,-21.50593,2021-04-12T09:32:32.635Z,0.039818,1.202524e+06,2017-10-19T00:00:00.000Z,None,2021-04-18T23:30:40.326Z
3,3,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,1.42,6.568149e+10,4,NaN,2.080601e+10,1.57,1.190000,-0.143111,-9.14432,-6.747876e+09,-9.31649,4.597332e+10,1.000000e+11,NaN,3.40,-57.96086,2018-01-07T00:00:00.000Z,0.002686,5.308576e+04,2014-05-22T00:00:00.000Z,None,2021-04-18T23:31:24.707Z
4,4,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,1.00,4.821533e+10,5,NaN,2.315707e+11,1.02,0.993047,-0.000765,-0.07641,-3.686910e+07,-0.07641,4.818719e+10,4.818719e+10,NaN,1.32,-24.37553,2018-07-24T00:00:00.000Z,0.572521,7.476806e+01,2015-03-02T00:00:00.000Z,None,2021-04-18T23:06:57.015Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6743,243,gneiss,gneiss,Gneiss,https://assets.coingecko.com/coins/images/1335...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+09,NaN,NaN,0.00000,None,NaN,0.000000e+00,None,None,None
6744,244,imusd,imusd,imUSD,https://assets.coingecko.com/coins/images/1369...,NaN,NaN,None,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,None,NaN,0.000000e+00,None,None,2021-03-14T02:49:13.557Z
6745,245,street-credit,cred,Street Credit,https://assets.coingecko.com/coins/images/1325...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.559600e+04,NaN,NaN,0.00000,None,NaN,0.000000e+00,None,None,None
6746,246,klondike-bond,kbond,Klondike Bond,https://assets.coingecko.com/coins/images/1378...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.537026e+01,NaN,NaN,0.00000,None,NaN,0.000000e+00,None,None,None


In [7]:
len(marketsDf)

6748

In [8]:
btcSupply=marketsDf[marketsDf.symbol=="btc"].circulating_supply[0]
btcSupply

18685856.0

In [9]:
marketsDf["btc_relative_val"]=marketsDf.circulating_supply/btcSupply*marketsDf.current_price

In [10]:
marketsDf.sort_values(by=["btc_relative_val"], ascending=[0], inplace=True)

In [11]:
marketsDf[["symbol","btc_relative_val","current_price", "circulating_supply","market_cap"]][(marketsDf["btc_relative_val"]<20) & (marketsDf["btc_relative_val"]>10)]

,symbol,btc_relative_val,current_price,circulating_supply,market_cap
170,axs,19.636589,6.640000,5.526001e+07,369226317.0
169,kmd,19.588290,2.920000,1.253507e+08,369390799.0
172,btm,19.545591,0.218573,1.670957e+09,365298750.0
171,alpha,19.545448,1.460000,2.501530e+08,366056044.0
173,kin,19.491215,0.000233,1.561056e+12,362359138.0
...,...,...,...,...,...
261,powr,10.332989,0.449325,4.297129e+08,193070568.0
263,ern,10.245348,22.150000,8.643029e+06,191556355.0
264,prq,10.130699,1.810000,1.045861e+08,190064143.0
265,iris,10.083412,0.192657,9.779929e+08,188543190.0


In [12]:
mmDf=createDf(url="https://api.coingecko.com/api/v3/exchanges/binance/tickers?",pages=10)

In [13]:
import pandas as pd
binanceDf=pd.DataFrame([ row for row in mmDf.tickers])[["coin_id","last"]]


In [14]:
import plotly.express as px
fig = px.bar(marketsDf.head(200), x='symbol', y='btc_relative_val')
fig.show()

find coins that have not spiked historically beyond a certain threshold

In [15]:
def getUnixTime(dt):
  import time
  import datetime
  if(dt is None):
    return int(time.time())
  else:
    d = datetime.date(dt[0],dt[1],dt[2])
    return int(time.mktime(d.timetuple()))

In [16]:
def getDate(timestamp):
  from datetime import datetime
  #try:
  return datetime.utcfromtimestamp(timestamp)
  #except ValueError:
  #  return int(timestamp)


In [17]:
(getUnixTime([2015,1,1]),getUnixTime(None))

(1420070400, 1618788792)

In [18]:
getDate(1618547586)

datetime.datetime(2021, 4, 16, 4, 33, 6)

In [21]:
def getHistory(coin="bitcoin",fromUnixTime=1420070400, toUnixTime=getUnixTime(None)):
  import pandas as pd
  apiurl="https://api.coingecko.com/api/v3/coins/"+coin+"/market_chart/range?vs_currency=usd&from="+str(fromUnixTime)+"&to="+str(toUnixTime)
  coinData=readJsonLink(apiurl)
  coinDf=pd.DataFrame(coinData["prices"], columns=["unixtime","price"])
  coinDf["date"]=coinDf.apply(lambda x : getDate(x["unixtime"]/1000),axis=1)
  coinDf=coinDf[["date","price"]]
  import plotly.express as px
  fig = px.line(coinDf, x="date", y="price", title=coin)
  fig.show()
  return coinDf
  #return coinDf

In [76]:
df=getHistory("bitcoin")


In [1]:
!pip install tbats

     |████████████████████████████████| 51kB 4.2MB/s 
     |████████████████████████████████| 1.5MB 19.5MB/s 
     |████████████████████████████████| 9.5MB 33.1MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [5]:
!pip uninstall numpy
!pip install numpy

Uninstalling numpy-1.19.5:
  Would remove:
    /usr/bin/f2py
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.7
    /usr/local/lib/python3.7/dist-packages/numpy-1.19.5.dist-info/*
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libgfortran-2e0d59d6.so.5.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libopenblasp-r0-09e95953.3.13.so
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libquadmath-2d0c479f.so.0.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libz-eb09ad1d.so.1.2.3
    /usr/local/lib/python3.7/dist-packages/numpy/*
Proceed (y/n)? y
  Successfully uninstalled numpy-1.19.5
     |████████████████████████████████| 15.3MB 323kB/s 
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have 

In [77]:
df = df.set_index('date')
y = df['price']
y_to_train = y.iloc[:(len(y)-365)]
y_to_test = y.iloc[(len(y)-365):] # last year for testing

In [28]:
from tbats import TBATS, BATS
# Fit the model
estimator = TBATS(seasonal_periods=(7, 365.25))


In [78]:
model = estimator.fit(y_to_train)
# Forecast 365 days ahead
y_forecast = model.forecast(steps=365)

In [79]:
y_forecast

array([7035.58345892, 7035.58345892, 7035.58345892, 7035.58345892,
       7035.58345892, 7035.58345892, 7035.58345892, 7035.58345892,
       7035.58345892, 7035.58345892, 7035.58345892, 7035.58345892,
       7035.58345892, 7035.58345892, 7035.58345892, 7035.58345892,
       7035.58345892, 7035.58345892, 7035.58345892, 7035.58345892,
       7035.58345892, 7035.58345892, 7035.58345892, 7035.58345892,
       7035.58345892, 7035.58345892, 7035.58345892, 7035.58345892,
       7035.58345892, 7035.58345892, 7035.58345892, 7035.58345892,
       7035.58345892, 7035.58345892, 7035.58345892, 7035.58345892,
       7035.58345892, 7035.58345892, 7035.58345892, 7035.58345892,
       7035.58345892, 7035.58345892, 7035.58345892, 7035.58345892,
       7035.58345892, 7035.58345892, 7035.58345892, 7035.58345892,
       7035.58345892, 7035.58345892, 7035.58345892, 7035.58345892,
       7035.58345892, 7035.58345892, 7035.58345892, 7035.58345892,
       7035.58345892, 7035.58345892, 7035.58345892, 7035.58345

In [81]:
df1=pd.DataFrame(y_forecast)

In [82]:
df1["idx"]=df1.index

In [83]:
df1

,0,idx
0,7035.583459,0
1,7035.583459,1
2,7035.583459,2
3,7035.583459,3
4,7035.583459,4
...,...,...
360,7035.583459,360
361,7035.583459,361
362,7035.583459,362
363,7035.583459,363


In [84]:
def addDays(n=0):
  from datetime import datetime, timedelta
  #d1=datetime.today() + timedelta(days=1)
  d1 = (datetime.today()+ timedelta(days=n)).strftime("%Y-%m-%d")
  d1=datetime.strptime(d1, '%Y-%m-%d')
  #print(d1)
  return d1
addDays()



datetime.datetime(2021, 4, 19, 0, 0)

In [86]:
df1["date"]=df1["idx"].apply(addDays)

In [87]:
df1.columns=['price','idx', 'date']

In [88]:
fig = px.line(df1, x="date", y="price", title="Prediction")
fig.show()